In [4]:
# scrape_all_programs_final.py

import re
import json

catalog_file = "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalog_june_2025.txt"
output_file = "all_programs_scraped.json"

# --- Known colleges ---
known_colleges = [
    "School of Business",
    "Leavitt School of Health",
    "School of Technology",
    "School of Education"
]

# --- Patterns ---
header_text = "CCN Course Number Course Description CUs Term"
footer_pattern = re.compile(r'Total CUs:\s*\d+', re.IGNORECASE)
dept_course_regex = re.compile(r'^([A-Z]{2,5})\s+(\d{4})\s+([A-Z]{1,4})\s*(\d{1,4}[A-Z]?)\s+(.*?)\s+(\d+)\s+\d+$')

# --- Load file ---
with open(catalog_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

programs = []
course_code_set = set()
colleges_used = set()
current_college = "Unknown"

i = 0
while i < len(lines):
    line = lines[i].strip()

    # Update college if match
    for college in known_colleges:
        if (college in line or (college + " Programs") in line) and \
           "Admission" not in line and "Senior VP" not in line:
            current_college = college
            break

    # Found a CU footer → possible program block
    if footer_pattern.search(line):
        cu_total = int(re.search(r'\d+', line).group())

        # Look for © line → then valid program title
        j = i + 2  # skip footer + ©
        title_line = None
        while j < len(lines) and not lines[j].startswith(header_text):
            candidate = lines[j].strip()
            if any(candidate.startswith(kw) for kw in ["Bachelor", "Master", "Certificate:", "Endorsement"]):
                title_line = candidate
                break
            j += 1

        if not title_line:
            title_line = "UNNAMED_PROGRAM"

        # Collect description
        desc_lines = []
        j += 1
        while j < len(lines) and not lines[j].startswith(header_text):
            desc_lines.append(lines[j].strip())
            j += 1
        description = " ".join(desc_lines).strip()

        # Collect courses
        courses = []
        j += 1  # skip header
        while j < len(lines) and not footer_pattern.search(lines[j]):
            cline = lines[j].strip()
            j += 1

            if not cline or cline.startswith(header_text):
                continue

            match = dept_course_regex.match(cline)
            if match:
                dept, num, wgu_prefix, wgu_code, name, cu = match.groups()
                course_code = f"{wgu_prefix}{wgu_code}"
                term = int(cline.split()[-1])
                courses.append({
                    "dept": dept,
                    "course_code": course_code,
                    "course_name": name.strip(),
                    "cu": int(cu),
                    "term": term
                })
                course_code_set.add(course_code)
            else:
                # fallback: loose format
                parts = cline.split()
                if len(parts) >= 3 and parts[-1].isdigit() and parts[-2].isdigit():
                    cu = int(parts[-2])
                    term = int(parts[-1])
                    name = " ".join(parts[:-2])
                    courses.append({
                        "dept": "N/A",
                        "course_code": "N/A",
                        "course_name": name.strip(),
                        "cu": cu,
                        "term": term
                    })

        programs.append({
            "college": current_college,
            "program_title": title_line,
            "program_description": description,
            "cu_total": cu_total,
            "courses": courses
        })
        colleges_used.add(current_college)
        i = j  # jump forward

    else:
        i += 1

# Save JSON
with open(output_file, "w", encoding="utf-8") as out:
    json.dump(programs, out, indent=2)

print("\n--- FINAL SCRAPE SUMMARY ---")
print(f"✅ Total programs scraped: {len(programs)}")
print(f"✅ Unique course codes: {len(course_code_set)}")
print(f"✅ Colleges detected: {sorted(colleges_used)}")
print(f"📄 Output: {output_file}")
print("-----------------------------\n")


--- FINAL SCRAPE SUMMARY ---
✅ Total programs scraped: 118
✅ Unique course codes: 705
✅ Colleges detected: ['School of Business']
📄 Output: all_programs_scraped.json
-----------------------------



In [ ]:
## debug catching college names


In [21]:
# scrape_all_programs_debug_colleges.py

import re
from collections import defaultdict

catalog_file = "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2025.txt"

tenets_pattern = re.compile(r'^(School of [A-Za-z ]+|Leavitt School of Health|Teachers College) Tenets:')

with open(catalog_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

college_blocks = []

for idx, line in enumerate(lines):
    match = tenets_pattern.match(line)
    if match:
        raw_name = match.group(1)
        if raw_name == "Teachers College":
            college_name = "School of Education"
        else:
            college_name = raw_name
        college_blocks.append((idx, college_name))

print("\n📌 Tenets Anchors Found:")
for idx, name in college_blocks:
    print(f"{idx} → {name}")

print(f"\n✅ Total colleges detected: {len(college_blocks)}\n")


📌 Tenets Anchors Found:
2667 → School of Business
3581 → Leavitt School of Health
4773 → School of Technology
5870 → School of Education

✅ Total colleges detected: 4



In [22]:
# scrape_all_programs_debug_college_descriptions.py

import re

catalog_file = "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2025.txt"

tenets_pattern = re.compile(r'^(School of [A-Za-z ]+|Leavitt School of Health|Teachers College) Tenets:')
copyright_pattern = re.compile(r'^© Western Governors University')

with open(catalog_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

college_blocks = []

# Find Tenets anchors
for idx, line in enumerate(lines):
    match = tenets_pattern.match(line)
    if match:
        raw_name = match.group(1)
        if raw_name == "Teachers College":
            college_name = "School of Education"
        else:
            college_name = raw_name
        college_blocks.append((idx, college_name))

print("\n📌 Extracted College Descriptions:\n")

for start_idx, college_name in college_blocks:
    desc_lines = []
    i = start_idx + 1  # skip Tenets line
    while i < len(lines) and not copyright_pattern.match(lines[i]):
        desc_lines.append(lines[i])
        i += 1
    description = " ".join(desc_lines).strip()
    print(f"{college_name} — {len(description.split())} words")
    print(f"  Preview: {description[:120]}...")
    print("-" * 50)

print(f"\n✅ Total colleges with description: {len(college_blocks)}\n")


📌 Extracted College Descriptions:

School of Business — 129 words
  Preview: • Impact: We are a global force for good; our shared purpose is to improve the lives of people and society through a tra...
--------------------------------------------------
Leavitt School of Health — 304 words
  Preview: • Each student is unique. We align our systems and resources using Learner-Centered Faculty (LCF) tools to support each ...
--------------------------------------------------
School of Technology — 295 words
  Preview: • We believe education is the single greatest catalyst to change lives permanently. By providing affordable, high- value...
--------------------------------------------------
School of Education — 550 words
  Preview: • WGU’s Teachers College radically improves the way people learn and lead across the K-20+ spectrum—i.e., the K- 12, hig...
--------------------------------------------------

✅ Total colleges with description: 4



In [30]:
# scrape_all_programs_debug_program_titles_fixed.py

import re
from collections import defaultdict

catalog_file = "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2025.txt"

tenets_pattern = re.compile(r'^(School of [A-Za-z ]+|Leavitt School of Health|Teachers College) Tenets:')
copyright_pattern = re.compile(r'^© Western Governors University')
footer_pattern = re.compile(r'Total CUs:\s*\d+', re.IGNORECASE)

title_pattern = re.compile(
    r'^(Bachelor|Master|B\.S\.|B\.A\.|M\.S\.|M\.A\.|MBA|Certificate:|Post-Master\'s Certificate|Endorsement)',
    re.IGNORECASE
)

with open(catalog_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

# Find college blocks
college_blocks = []
for idx, line in enumerate(lines):
    match = tenets_pattern.match(line)
    if match:
        raw_name = match.group(1)
        college_name = "School of Education" if raw_name == "Teachers College" else raw_name
        college_blocks.append((idx, college_name))

# Add end bounds
college_blocks_with_end = []
for n, (start_idx, college_name) in enumerate(college_blocks):
    end_idx = college_blocks[n + 1][0] if n + 1 < len(college_blocks) else len(lines)
    college_blocks_with_end.append((start_idx, end_idx, college_name))

college_programs = defaultdict(list)

for start, end, college_name in college_blocks_with_end:
    i = start + 1

    while i < end:
        if copyright_pattern.match(lines[i]):
            i += 1  # next line should be the program title
            if i < end:
                title_candidate = lines[i]
                if title_pattern.match(title_candidate):
                    college_programs[college_name].append(title_candidate)
            i += 1
            continue

        if footer_pattern.search(lines[i]):
            i += 1
            continue

        i += 1

print("\n📌 College → Program Titles:\n")
for college, titles in college_programs.items():
    print(f"{college} ({len(titles)} programs)")
    for t in titles:
        print(f"  - {t}")
    print("-" * 60)

print("\n✅ Done — programs grouped and listed.\n")


📌 College → Program Titles:

School of Business (21 programs)
  - Bachelor of Science, Accounting
  - Bachelor of Science, Human Resource Management
  - Bachelor of Science, Information Technology Management
  - Bachelor of Science, Business Management
  - Bachelor of Science, Marketing
  - Bachelor of Science, Communications
  - Bachelor of Science, Finance
  - Bachelor of Science, Healthcare Administration
  - Bachelor of Science, Supply Chain and Operations Management
  - Bachelor of Science, User Experience Design
  - Master of Business Administration
  - MBA, IT Management
  - Master of Business Administration, Healthcare Administration
  - Master of Science, Management and Leadership
  - Master of Science in Marketing, Digital Marketing Specialization
  - Master of Science in Marketing, Marketing Analytics Specialization
  - Master of Science in Accounting, Auditing Specialization
  - Master of Science in Accounting, Financial Reporting Specialization
  - Master of Science in Ac

In [32]:
# scrape_all_programs_debug_program_titles_fixed.py

import re
from collections import defaultdict

catalog_file = "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2025.txt"

tenets_pattern = re.compile(r'^(School of [A-Za-z ]+|Leavitt School of Health|Teachers College) Tenets:')
certificates_header_pattern = re.compile(r'^Certificates - Standard Paths')
copyright_pattern = re.compile(r'^© Western Governors University')
footer_pattern = re.compile(r'Total CUs:\s*\d+', re.IGNORECASE)

title_pattern = re.compile(
    r'^(Bachelor|Master|B\.S\.|B\.A\.|M\.S\.|M\.A\.|MBA|Certificate:|Post-Master\'s Certificate|Endorsement)',
    re.IGNORECASE
)

with open(catalog_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

# Find college blocks
college_blocks = []
cert_header_idx = None

for idx, line in enumerate(lines):
    if certificates_header_pattern.match(line):
        cert_header_idx = idx
        break  # Stop scanning for colleges

    match = tenets_pattern.match(line)
    if match:
        raw_name = match.group(1)
        college_name = "School of Education" if raw_name == "Teachers College" else raw_name
        college_blocks.append((idx, college_name))

# Add end bounds for each college block, stop at Certificates section if present
college_blocks_with_end = []
for n, (start_idx, college_name) in enumerate(college_blocks):
    if n + 1 < len(college_blocks):
        end_idx = college_blocks[n + 1][0]
    elif cert_header_idx:
        end_idx = cert_header_idx
    else:
        end_idx = len(lines)
    college_blocks_with_end.append((start_idx, end_idx, college_name))

college_programs = defaultdict(list)

for start, end, college_name in college_blocks_with_end:
    i = start + 1

    while i < end:
        if copyright_pattern.match(lines[i]):
            i += 1  # next line should be the program title
            if i < end:
                title_candidate = lines[i]
                if title_pattern.match(title_candidate):
                    college_programs[college_name].append(title_candidate)
            i += 1
            continue

        if footer_pattern.search(lines[i]):
            i += 1
            continue

        i += 1

print("\n📌 College → Program Titles:\n")
for college, titles in college_programs.items():
    print(f"{college} ({len(titles)} programs)")
    for t in titles:
        print(f"  - {t}")
    print("-" * 60)

print("\n✅ Done — colleges grouped and titles listed.\n")


📌 College → Program Titles:

School of Business (21 programs)
  - Bachelor of Science, Accounting
  - Bachelor of Science, Human Resource Management
  - Bachelor of Science, Information Technology Management
  - Bachelor of Science, Business Management
  - Bachelor of Science, Marketing
  - Bachelor of Science, Communications
  - Bachelor of Science, Finance
  - Bachelor of Science, Healthcare Administration
  - Bachelor of Science, Supply Chain and Operations Management
  - Bachelor of Science, User Experience Design
  - Master of Business Administration
  - MBA, IT Management
  - Master of Business Administration, Healthcare Administration
  - Master of Science, Management and Leadership
  - Master of Science in Marketing, Digital Marketing Specialization
  - Master of Science in Marketing, Marketing Analytics Specialization
  - Master of Science in Accounting, Auditing Specialization
  - Master of Science in Accounting, Financial Reporting Specialization
  - Master of Science in Ac

In [33]:
# scrape_all_programs_debug_program_titles_fixed.py

import re
from collections import defaultdict

catalog_file = "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2025.txt"

tenets_pattern = re.compile(r'^(School of [A-Za-z ]+|Leavitt School of Health|Teachers College) Tenets:')
certificates_header_pattern = re.compile(r'^Certificates - Standard Paths')
copyright_pattern = re.compile(r'^© Western Governors University')
footer_pattern = re.compile(r'Total CUs:\s*\d+', re.IGNORECASE)

title_pattern = re.compile(
    r'^(Bachelor|Master|B\.S\.|B\.A\.|M\.S\.|M\.A\.|MBA|Certificate:|Post-Master\'s Certificate|Endorsement)',
    re.IGNORECASE
)

with open(catalog_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

# Find college blocks
college_blocks = []
cert_header_idx = None

for idx, line in enumerate(lines):
    if certificates_header_pattern.match(line):
        cert_header_idx = idx
        break  # Stop scanning for colleges

    match = tenets_pattern.match(line)
    if match:
        raw_name = match.group(1)
        college_name = "School of Education" if raw_name == "Teachers College" else raw_name
        college_blocks.append((idx, college_name))

# Add end bounds for each college block, stop at Certificates section if present
college_blocks_with_end = []
for n, (start_idx, college_name) in enumerate(college_blocks):
    if n + 1 < len(college_blocks):
        end_idx = college_blocks[n + 1][0]
    elif cert_header_idx:
        end_idx = cert_header_idx
    else:
        end_idx = len(lines)
    college_blocks_with_end.append((start_idx, end_idx, college_name))

college_programs = defaultdict(list)

for start, end, college_name in college_blocks_with_end:
    i = start + 1

    while i < end:
        if copyright_pattern.match(lines[i]):
            i += 1  # next line should be the program title
            if i < end:
                title_candidate = lines[i]
                if title_pattern.match(title_candidate):
                    college_programs[college_name].append(title_candidate)
            i += 1
            continue

        if footer_pattern.search(lines[i]):
            i += 1
            continue

        i += 1

print("\n📌 College → Program Titles (First Example Only):\n")
for college, titles in college_programs.items():
    print(f"{college} ({len(titles)} programs)")
    if titles:
        print(f"  - {titles[0]}")
    print("-" * 60)

print("\n✅ Done — colleges grouped, count and first example shown.\n")


📌 College → Program Titles (First Example Only):

School of Business (21 programs)
  - Bachelor of Science, Accounting
------------------------------------------------------------
Leavitt School of Health (22 programs)
  - Bachelor of Science, Nursing - Prelicensure (Pre-Nursing)
------------------------------------------------------------
School of Technology (25 programs)
  - Bachelor of Science, Cloud Computing - Amazon Web Services track
------------------------------------------------------------
School of Education (46 programs)
  - Bachelor of Arts, Elementary Education
------------------------------------------------------------

✅ Done — colleges grouped, count and first example shown.



In [35]:
## test all years

In [34]:
# scrape_all_programs_debug_all_catalogs.py

import re
from collections import defaultdict

catalog_files = [
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_july_2017.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2018.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2019.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2020.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2021.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2022.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2023.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2024.txt",
    "/Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2025.txt"
]

tenets_pattern = re.compile(r'^(School of [A-Za-z ]+|Leavitt School of Health|Teachers College) Tenets:')
certificates_header_pattern = re.compile(r'^Certificates - Standard Paths')
copyright_pattern = re.compile(r'^© Western Governors University')
footer_pattern = re.compile(r'Total CUs:\s*\d+', re.IGNORECASE)

title_pattern = re.compile(
    r'^(Bachelor|Master|B\.S\.|B\.A\.|M\.S\.|M\.A\.|MBA|Certificate:|Post-Master\'s Certificate|Endorsement)',
    re.IGNORECASE
)

for catalog_file in catalog_files:
    with open(catalog_file, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f]

    college_blocks = []
    cert_header_idx = None

    for idx, line in enumerate(lines):
        if certificates_header_pattern.match(line):
            cert_header_idx = idx
            break

        match = tenets_pattern.match(line)
        if match:
            raw_name = match.group(1)
            college_name = "School of Education" if raw_name == "Teachers College" else raw_name
            college_blocks.append((idx, college_name))

    college_blocks_with_end = []
    for n, (start_idx, college_name) in enumerate(college_blocks):
        if n + 1 < len(college_blocks):
            end_idx = college_blocks[n + 1][0]
        elif cert_header_idx:
            end_idx = cert_header_idx
        else:
            end_idx = len(lines)
        college_blocks_with_end.append((start_idx, end_idx, college_name))

    college_programs = defaultdict(list)

    for start, end, college_name in college_blocks_with_end:
        i = start + 1

        while i < end:
            if copyright_pattern.match(lines[i]):
                i += 1
                if i < end:
                    title_candidate = lines[i]
                    if title_pattern.match(title_candidate):
                        college_programs[college_name].append(title_candidate)
                i += 1
                continue

            if footer_pattern.search(lines[i]):
                i += 1
                continue

            i += 1

    print(f"\n📌 Catalog: {catalog_file}")
    for college, titles in college_programs.items():
        print(f"{college} ({len(titles)} programs)")
        if titles:
            print(f"  - {titles[0]}")
        print("-" * 60)
    print("\n✅ Done — colleges grouped, count and first example shown.\n")


📌 Catalog: /Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_july_2017.txt

✅ Done — colleges grouped, count and first example shown.


📌 Catalog: /Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2018.txt

✅ Done — colleges grouped, count and first example shown.


📌 Catalog: /Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2019.txt

✅ Done — colleges grouped, count and first example shown.


📌 Catalog: /Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2020.txt

✅ Done — colleges grouped, count and first example shown.


📌 Catalog: /Users/buddy/Desktop/WGU-Reddit/data/WGU_catalog/catalogs-2017-2025/catalog_june_2021.txt
School of Education (1 programs)
  - Bachelor of Arts, Elementary Education
------------------------------------------------------------

✅ Done — colleges grouped, count and first example shown.


📌 Catalog: /Users/buddy/Desktop/WGU-Reddit/data/WGU_

In [24]:
# list_programs_school_of_business.py

college_name = "School of Business"

print(f"\n📌 Programs in {college_name}:\n")
for idx, title in enumerate(college_programs.get(college_name, []), 1):
    print(f"{idx:2d}. {title}")

print(f"\n✅ Total: {len(college_programs.get(college_name, []))}\n")


📌 Programs in School of Business:

 1. Bachelor of Science, Accounting
 2. Bachelor of Science, Human Resource Management
 3. Bachelor of Science, Information Technology Management
 4. Bachelor of Science, Business Management
 5. Bachelor of Science, Marketing
 6. Bachelor of Science, Communications
 7. Bachelor of Science, Finance
 8. B.S. Finance will prepare graduates for the challenges of the profession by focusing on five key areas: Personal
 9. Bachelor of Science, Healthcare Administration
10. Bachelor of Science, Supply Chain and Operations Management
11. Bachelor of Science, User Experience Design
12. Master of Business Administration
13. MBA 201404 Total CUs: 34
14. MBA, IT Management
15. MBAITM 201408 Total CUs: 35
16. Master of Business Administration, Healthcare Administration
17. MBAHA 202507 Total CUs: 36
18. Master of Science, Management and Leadership
19. Master of Science in Marketing, Digital Marketing Specialization
20. Master of Science in Marketing, Marketing Ana

In [25]:
# list_programs_leavitt_school_of_health.py

college_name = "Leavitt School of Health"

print(f"\n📌 Programs in {college_name}:\n")
for idx, title in enumerate(college_programs.get(college_name, []), 1):
    print(f"{idx:2d}. {title}")

print(f"\n✅ Total: {len(college_programs.get(college_name, []))}\n")


📌 Programs in Leavitt School of Health:

 1. Bachelor of Science, Nursing - Prelicensure (Pre-Nursing)
 2. Bachelor of Science, Nursing - Prelicensure (Nursing)
 3. Bachelor of Science, Nursing
 4. Bachelor of Science, Health Information Management
 5. Bachelor of Science program consists of the following areas of study: Healthcare Data, Health Information
 6. Bachelor of Science, Health and Human Services
 7. Bachelor of Science, Health Science
 8. Bachelor of Science in Psychology
 9. Bachelor of Science, Public Health
10. Master of Science in Public Health (MSPH) degree at WGU. To facilitate this transition, bridge courses in leadership
11. Master of Science, Nursing - Family Nurse Practitioner (BSN to MSN)
12. Master of Science, Nursing - Psychiatric Mental Health Nurse Practitioner
13. Master of Science, Nursing - Education (BSN to MSN)
14. Master of Science, Nursing - Leadership and Management (BSN to MSN)
15. Master of Science, Nursing - Leadership and Management program conten

In [26]:
# list_programs_school_of_technology.py

college_name = "School of Technology"

print(f"\n📌 Programs in {college_name}:\n")
for idx, title in enumerate(college_programs.get(college_name, []), 1):
    print(f"{idx:2d}. {title}")

print(f"\n✅ Total: {len(college_programs.get(college_name, []))}\n")


📌 Programs in School of Technology:

 1. Bachelor of Science, Cloud Computing - Amazon Web Services track
 2. Bachelor of Science, Cloud Computing - Microsoft Azure track
 3. Bachelor of Science, Cloud Computing
 4. Bachelor of Science, Computer Science
 5. Bachelor of Science, Computer Science (BSCS to MSCS)
 6. Bachelor of Science, Cybersecurity and Information Assurance
 7. Bachelor of Science, Data Analytics
 8. Bachelor of Science, Information Technology
 9. Bachelor of Science, Information Technology (BSIT to MSITM)
10. Master of Science, Information Technology Management portion
11. Bachelor of Science, Network Engineering and Security Cisco Track
12. Bachelor of Science, Software Engineering
13. Bachelor of Science, Software Engineering
14. Bachelor of Science, Software Engineering (BSSWE to MSSWE)
15. bachelors and bridge course work, will receive the Bachelor of Science, Software Engineering degree. Students can
16. Master of Science, Computer Science, Artificial Intelligenc

In [27]:
# list_programs_school_of_education.py

college_name = "School of Education"

print(f"\n📌 Programs in {college_name}:\n")
for idx, title in enumerate(college_programs.get(college_name, []), 1):
    print(f"{idx:2d}. {title}")

print(f"\n✅ Total: {len(college_programs.get(college_name, []))}\n")


📌 Programs in School of Education:

 1. Bachelor of Arts, Elementary Education
 2. Bachelor of Arts, Special Education and Elementary Education (Dual Licensure)
 3. Bachelor of Arts, Special Education, Mild to Moderate
 4. Bachelor of Science, Mathematics Education (Secondary)
 5. Bachelor of Science, Science Education (Secondary Biological Science)
 6. Bachelor of Science, Science Education (Secondary Chemistry)
 7. Bachelor of Science, Science Education (Secondary Earth Science)
 8. Bachelor of Science, Science Education (Secondary Physics)
 9. Bachelor of Arts, Educational Studies in Elementary Education
10. Bachelor of Arts, Educational Studies in Special and Elementary Education
11. Bachelor of Arts, Educational Studies in Mild to Moderate Exceptionalities Special Education
12. Bachelor of Arts, Educational Studies in Secondary Mathematics Education
13. Bachelor of Arts, Educational Studies in Secondary Biological Science Education
14. Bachelor of Arts, Educational Studies in Sec

In [3]:
# Show all unique program titles

unique_programs = df["program_title"].unique()

print(f"Total unique programs: {len(unique_programs)}\n")
for title in unique_programs:
    print("-", title)

Total unique programs: 121

- Bachelor of Science, Human Resource Management
- Bachelor of Science, Information Technology Management
- Bachelor of Science, Business Management
- Bachelor of Science, Marketing
- Bachelor of Science, Communications
- Bachelor of Science, Finance
- Bachelor of Science, Healthcare Administration
- Bachelor of Science, Supply Chain and Operations Management
- Bachelor of Science, User Experience Design
- Master of Business Administration
- MBA, IT Management
- Master of Business Administration, Healthcare Administration
- Master of Science, Management and Leadership
- Master of Science in Marketing, Digital Marketing Specialization
- Master of Science in Marketing, Marketing Analytics Specialization
- Master of Science in Accounting, Auditing Specialization
- Master of Science in Accounting, Financial Reporting Specialization
- Master of Science in Accounting, Management Accounting Specialization
- Master of Science in Accounting, Taxation Specialization
-